In [48]:
%pip install yfinance


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
import yfinance as yf

In [50]:
data = yf.download(['MSFT', 'AAPL', 'GOOG'], period='1mo')
changes = data.Close.pct_change()
mean = changes.mean()
# ddof of 1 because its a sample of a population
sd = changes.std(ddof=1).dropna()
print("changes")
print(changes.tail())

print("mean")
print(mean)

print("standard deviation")
print(sd)

[*********************100%***********************]  3 of 3 completed

changes
Ticker          AAPL      GOOG      MSFT
Date                                    
2025-01-27  0.031780 -0.040267 -0.021394
2025-01-28  0.036544  0.017031  0.029087
2025-01-29  0.004617  0.000558 -0.010890
2025-01-30 -0.007395  0.027640 -0.061809
2025-01-31 -0.006692  0.014657  0.000169
mean
Ticker
AAPL   -0.001551
GOOG    0.004126
MSFT   -0.000221
dtype: float64
standard deviation
Ticker
AAPL    0.018935
GOOG    0.017144
MSFT    0.021611
dtype: float64


In [51]:
z_scores = (changes - mean) / sd
z_scores

Ticker,AAPL,GOOG,MSFT
Date,,,
2025-01-02,NaN,NaN,NaN
2025-01-03,-0.024233,0.524269,0.537508
2025-01-06,0.437780,1.218082,0.502054
2025-01-07,-0.519509,-0.609011,-0.582457
2025-01-08,0.188724,-0.632110,0.250128
2025-01-10,-1.191056,-0.903430,-0.601215
2025-01-13,-0.464396,-0.506420,-0.184183
2025-01-14,-0.170450,-0.616835,-0.158381
2025-01-15,1.120991,1.569804,1.194638


In [52]:
bad_data = z_scores[z_scores > 2]
bad_data.dropna(how="all")

Ticker,AAPL,GOOG,MSFT
Date,,,
2025-01-28,2.011808,NaN,NaN


In [53]:
Q1 = changes.quantile(0.10)
Q3 = changes.quantile(0.90)
IQR = Q3 - Q1

# Define outliers as values outside 1.5 * IQR from Q1 or Q3
outliers = (changes < (Q1 - 1.5 * IQR)) | (changes > (Q3 + 1.5 * IQR))
print("Outliers (IQR Method):")
print(outliers.tail())

Outliers (IQR Method):
Ticker       AAPL   GOOG   MSFT
Date                           
2025-01-27  False  False  False
2025-01-28  False  False  False
2025-01-29  False  False  False
2025-01-30  False  False  False
2025-01-31  False  False  False


In [54]:
cleaned_data = data.copy()
cleaned_data[outliers] = None
cleaned_data = cleaned_data.interpolate(method='linear')
cleaned_data

Price            Close                                High              \
Ticker            AAPL        GOOG        MSFT        AAPL        GOOG   
Date                                                                     
2025-01-02  243.850006  190.630005  418.579987  249.100006  193.199997   
2025-01-03  243.360001  193.130005  423.350006  244.179993  194.500000   
2025-01-06  245.000000  197.960007  427.850006  247.330002  199.559998   
2025-01-07  242.210007  196.710007  422.369995  245.550003  202.139999   
2025-01-08  242.699997  195.389999  424.559998  243.710007  197.639999   
2025-01-10  236.850006  193.169998  418.950012  240.160004  197.619995   
2025-01-13  234.399994  192.289993  417.190002  234.669998  192.490005   
2025-01-14  233.279999  191.050003  415.670013  236.119995  193.270004   
2025-01-15  237.869995  196.979996  426.309998  238.960007  197.800003   
2025-01-16  228.259995  194.410004  424.579987  238.009995  196.979996   
2025-01-17  229.979996  197.550003  429.029999  232.289993  198.809998   
2025-01-21  222.639999  199.630005  428.500000  224.419998  203.839996   
2025-01-22  223.830002  200.029999  446.200012  224.119995  202.119995   
2025-01-23  223.660004  199.580002  446.709991  227.029999  201.940002   
2025-01-24  222.779999  201.899994  444.059998  225.630005  202.570007   
2025-01-27  229.860001  193.770004  434.559998  232.149994  198.669998   
2025-01-28  238.259995  197.070007  447.200012  240.190002  197.229996   
2025-01-29  239.360001  197.179993  442.329987  239.860001  198.460007   
2025-01-30  237.589996  202.630005  414.989990  240.789993  203.238007   
2025-01-31  236.000000  205.600006  415.059998  247.190002  207.080002   

Price                          Low                                Open  \
Ticker            MSFT        AAPL        GOOG        MSFT        AAPL   
Date                                                                     
2025-01-02  426.070007  241.820007  188.710007  414.850006  248.929993   
2025-01-03  424.029999  241.889999  191.350006  419.540009  243.360001   
2025-01-06  434.320007  243.199997  195.059998  425.480011  244.309998   
2025-01-07  430.649994  241.350006  195.940002  420.799988  242.979996   
2025-01-08  426.970001  240.050003  193.750000  421.540009  241.919998   
2025-01-10  424.709991  233.000000  191.600006  415.019989  240.009995   
2025-01-13  418.500000  229.720001  188.660004  412.290009  233.529999   
2025-01-14  419.739990  232.470001  189.639999  410.720001  234.750000   
2025-01-15  428.149994  234.429993  193.330002  418.269989  234.639999   
2025-01-16  429.489990  228.029999  194.300003  424.390015  237.350006   
2025-01-17  434.480011  228.479996  195.309998  428.170013  232.119995   
2025-01-21  430.899994  219.380005  199.440002  425.600006  224.000000   
2025-01-22  447.269989  219.789993  199.199997  436.000000  219.789993   
2025-01-23  446.750000  222.300003  196.820007  441.500000  224.740005   
2025-01-24  446.649994  221.410004  199.783997  441.399994  224.779999   
2025-01-27  435.200012  223.979996  192.699997  423.500000  224.020004   
2025-01-28  448.380005  230.809998  192.610001  431.380005  230.850006   
2025-01-29  446.880005  234.009995  195.190002  440.399994  234.119995   
2025-01-30  422.859985  237.210007  199.472000  413.160004  238.669998   
2025-01-31  420.690002  233.440002  203.580002  414.910004  247.190002   

Price                                  Volume                      
Ticker            GOOG        MSFT       AAPL      GOOG      MSFT  
Date                                                               
2025-01-02  191.485001  425.529999   55740700  17545200  16896500  
2025-01-03  192.725006  421.079987   40244100  12875000  16662900  
2025-01-06  195.149994  428.000000   45045600  19483300  20573600  
2025-01-07  198.270004  429.000000   40856000  16966800  18139100  
2025-01-08  193.949997  423.459991   37628900  14335300  15054600  
2025-01-10  195.419998  424.630005   61710900  207